<a href="https://colab.research.google.com/github/RAVITEJA-VADLURI/Mishra-sir-s-assignments/blob/main/asgn(3)_Monte_Carlo(2303A51942).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Implementing Monte Carlo Methods for Policy Evaluation and Control
import numpy as np
import random

N = 5
actions = [(0,1),(0,-1),(1,0),(-1,0)]  # Right, Left, Down, Up
action_labels = ["R","L","D","U"]
goal, pit = (4,4), (2,2)
gamma = 0.9

def reward(state):
    if state == goal: return 10
    if state == pit: return -10
    return -1

def next_state(state, action):
    if state in [goal,pit]:
        return state
    x,y = state
    dx,dy = action
    nx,ny = x+dx, y+dy
    if 0 <= nx < N and 0 <= ny < N:
        return (nx,ny)
    return state  # invalid -> stay

# Generate an episode following policy
def generate_episode(policy, max_steps=50):
    episode = []
    state = (0,0)  # start
    for _ in range(max_steps):
        if state in [goal,pit]:
            break
        action_idx = policy[state]
        action = actions[action_idx]
        next_s = next_state(state, action)
        r = reward(next_s)
        episode.append((state, action_idx, r))
        state = next_s
    return episode

# Monte Carlo Policy Evaluation
def mc_policy_evaluation(policy, episodes=5000):
    returns_sum = {}
    returns_count = {}
    V = np.zeros((N,N))

    for _ in range(episodes):
        ep = generate_episode(policy)
        G = 0
        visited = set()
        for t in reversed(range(len(ep))):
            s,a,r = ep[t]
            G = gamma*G + r
            if s not in visited:
                visited.add(s)
                returns_sum[s] = returns_sum.get(s,0)+G
                returns_count[s] = returns_count.get(s,0)+1
                V[s] = returns_sum[s]/returns_count[s]
    return V

# Monte Carlo Control with epsilon-greedy
def mc_control_epsilon_greedy(episodes=10000, epsilon=0.1):
    Q = np.zeros((N,N,len(actions)))
    returns_sum, returns_count = {}, {}
    policy = np.random.choice(len(actions), size=(N,N))

    for _ in range(episodes):
        # ε-greedy action selection
        def choose_action(s):
            if random.random() < epsilon:
                return np.random.choice(len(actions))
            else:
                return np.argmax(Q[s])

        # generate episode
        episode=[]
        state=(0,0)
        for t in range(50):
            if state in [goal,pit]:
                break
            a=choose_action(state)
            next_s=next_state(state,actions[a])
            r=reward(next_s)
            episode.append((state,a,r))
            state=next_s

        # update returns
        G=0
        visited=set()
        for t in reversed(range(len(episode))):
            s,a,r=episode[t]
            G=gamma*G+r
            if (s,a) not in visited:
                visited.add((s,a))
                returns_sum[(s,a)]=returns_sum.get((s,a),0)+G
                returns_count[(s,a)]=returns_count.get((s,a),0)+1
                Q[s][a]=returns_sum[(s,a)]/returns_count[(s,a)]
                policy[s]=np.argmax(Q[s])  # greedy improvement

    return Q, policy

# ---------------- Run ----------------
# Fixed policy: always Right if possible, else Down
fixed_policy = np.zeros((N,N),dtype=int)  # all "Right" by default
for i in range(N):
    for j in range(N):
        if j==N-1: fixed_policy[i,j]=2  # Down if at right border

V_mc = mc_policy_evaluation(fixed_policy, episodes=5000)
print("Monte Carlo Policy Evaluation (V):")
print(np.round(V_mc,2))

Q_mc, pi_mc = mc_control_epsilon_greedy(episodes=10000)
print("\nMonte Carlo Control Policy:")
print(pi_mc)


Monte Carlo Policy Evaluation (V):
[[-0.43  0.63  1.81  3.12  4.58]
 [ 0.    0.    0.    0.    6.2 ]
 [ 0.    0.    0.    0.    8.  ]
 [ 0.    0.    0.    0.   10.  ]
 [ 0.    0.    0.    0.    0.  ]]

Monte Carlo Control Policy:
[[2 2 0 0 2]
 [0 0 0 0 2]
 [2 2 2 2 2]
 [2 2 0 2 2]
 [0 0 0 0 1]]
